In [1]:
! pip install bilby[gw]  starccato-jax -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.9/102.9 kB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 54.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.2/40.2 MB 17.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.2/3.2 MB 58.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 315.5/315.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.8/43.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.1/131.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.7/182.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 56.

In [4]:
import os
import bilby
import numpy as np
import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from bilby.core.utils.random import seed

from starccato_jax.waveforms import StarccatoCCSNe

seed(123)
jax.config.update("jax_enable_x64", True)

# ----------------------------
# MODEL + GLOBAL CONFIGURATION
# ----------------------------
STARCCATO_MODEL = StarccatoCCSNe()
NUM_VAE_LATENTS = 32
LATENT_PARAMETER_NAMES = [f"z_{i}" for i in range(NUM_VAE_LATENTS)]
REFERENCE_DISTANCE_KPC = 10.0
REFERENCE_STRAIN_SCALE = 1e-21
SAMPLING_FREQUENCY = 4096.0

_W_EX = STARCCATO_MODEL.generate(rng=jax.random.PRNGKey(0), n=1)[0]
WAVEFORM_NUM_SAMPLES = int(np.asarray(_W_EX).shape[-1])
DURATION = WAVEFORM_NUM_SAMPLES / SAMPLING_FREQUENCY


# ----------------------------
# HELPER UTILITIES
# ----------------------------
def _to_numpy64(x):
    """Convert JAX/NumPy array to contiguous float64 NumPy array."""
    try:
        x = jax.device_get(x)
    except Exception:
        pass
    return np.asarray(x, dtype=np.float64)


# ----------------------------
# STARCCATO SUPERNOVA GENERATOR
# ----------------------------
def starccato_supernova(
    time_array,
    luminosity_distance,
    *,
    model: StarccatoCCSNe,
    latent_parameter_names,
    reference_distance=REFERENCE_DISTANCE_KPC,
    strain_scale=REFERENCE_STRAIN_SCALE,
    **parameters,
):
    latents = np.array(
        [parameters.get(name, 0.0) for name in latent_parameter_names],
        dtype=np.float32,
    ).reshape(1, -1)

    waveform = model.generate(z=latents)[0]
    waveform = _to_numpy64(waveform)

    target_len = int(np.asarray(time_array).size)
    if waveform.size == target_len:
        pass
    elif waveform.size == target_len + 1:
        waveform = waveform[:target_len]
    elif waveform.size + 1 == target_len:
        waveform = np.pad(waveform, (0, 1), mode="constant")
    else:
        raise ValueError(
            f"Waveform length ({waveform.size}) != time array length ({target_len})"
        )

    distance_scale = float(reference_distance) / float(luminosity_distance)
    intrinsic_amp = float(parameters.get("intrinsic_amplitude", 1.0))
    scaled = intrinsic_amp * float(strain_scale) * distance_scale * waveform

    plus = _to_numpy64(scaled)
    cross = plus.copy()
    return {"plus": plus, "cross": cross}


# ----------------------------
# PLOTTING UTILITIES
# ----------------------------
def plot_waveform_and_psd_comparison(
    ifos,
    time_array,
    waveform,
    outpath,
    title_prefix="",
    posterior_samples=None,
    waveform_generator=None,
    credible_interval=0.9,
):
    """
    Plot time-domain waveform and PSDs (noise, signal, data, posterior credible).
    """
    waveform = _to_numpy64(waveform)
    fig, axes = plt.subplots(1, len(ifos) + 1, figsize=(6 * (len(ifos) + 1), 4))

    # (1) Time-domain waveform
    ax0 = axes[0]
    ax0.plot(time_array, waveform, color="tab:orange")
    ax0.set_title(f"{title_prefix} waveform (time domain)")
    ax0.set_xlabel("Time [s]")
    ax0.set_ylabel("Strain")
    ax0.grid(True, alpha=0.3)

    # (2) PSD comparisons
    for i, ifo in enumerate(ifos, start=1):
        freqs = ifo.frequency_array
        psd = ifo.power_spectral_density_array

        hf = np.fft.rfft(waveform)
        freq_sig = np.fft.rfftfreq(len(time_array), d=1.0 / ifo.strain_data.sampling_frequency)
        sig_psd = (np.abs(hf) ** 2) / len(time_array)

        data = ifo.strain_data.time_domain_strain
        hf_data = np.fft.rfft(data)
        data_psd = (np.abs(hf_data) ** 2) / len(data)

        ax = axes[i]
        ax.loglog(freqs, psd, color="black", label="Noise PSD")
        ax.loglog(freq_sig, sig_psd, color="tab:orange", label="Signal PSD")
        ax.loglog(freq_sig, data_psd, color="tab:blue", alpha=0.7, label="Data PSD")

        # Posterior credible region
        if posterior_samples is not None and waveform_generator is not None:
            post_psds = []
            for _, sample in posterior_samples.iterrows():
                wf = waveform_generator.time_domain_strain(dict(sample))["plus"]
                hf_post = np.fft.rfft(wf)
                psd_post = (np.abs(hf_post) ** 2) / len(wf)
                post_psds.append(psd_post)

            post_psds = np.array(post_psds)
            lower = np.percentile(post_psds, (1 - credible_interval) / 2 * 100, axis=0)
            upper = np.percentile(post_psds, (1 + credible_interval) / 2 * 100, axis=0)
            median = np.median(post_psds, axis=0)

            ax.fill_between(
                freq_sig, lower, upper, color="tab:green", alpha=0.3, label=f"{int(credible_interval*100)}% CI"
            )
            ax.loglog(freq_sig, median, color="tab:green", alpha=0.8, label="Posterior median PSD")

        ax.set_title(f"{ifo.name} PSD comparison")
        ax.set_xlabel("Frequency [Hz]")
        ax.set_ylabel("PSD [1/Hz]")
        ax.legend()
        ax.grid(True, alpha=0.3)

    fig.tight_layout()
    fig.savefig(outpath, dpi=150)
    plt.close(fig)


# ----------------------------
# BILBY SETUP
# ----------------------------
duration = DURATION
sampling_frequency = SAMPLING_FREQUENCY
outdir = "outdir"
label = "supernova"
bilby.core.utils.setup_logger(outdir=outdir, label=label)
os.makedirs(outdir, exist_ok=True)

# Injection parameters
injection_parameters = dict(
    luminosity_distance=7.0,
    ra=4.6499,
    dec=-0.5063,
    geocent_time=1126259642.413,
    psi=2.659,
    intrinsic_amplitude=1.0,
)
for name in LATENT_PARAMETER_NAMES:
    injection_parameters[name] = 0.0

# Waveform generator
waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    time_domain_source_model=starccato_supernova,
    parameters=injection_parameters,
    parameter_conversion=lambda parameters: (parameters, list()),
    waveform_arguments=dict(
        model=STARCCATO_MODEL,
        latent_parameter_names=LATENT_PARAMETER_NAMES,
        reference_distance=REFERENCE_DISTANCE_KPC,
        strain_scale=REFERENCE_STRAIN_SCALE,
    ),
)

# Detectors
ifos = bilby.gw.detector.InterferometerList(["H1", "L1"])
ifos.set_strain_data_from_power_spectral_densities(
    sampling_frequency=sampling_frequency,
    duration=duration,
    start_time=injection_parameters["geocent_time"] - duration / 2,
)

# ----------------------------
# INITIAL PLOTS
# ----------------------------
preview_time = (
    np.arange(WAVEFORM_NUM_SAMPLES, dtype=np.float64) - WAVEFORM_NUM_SAMPLES / 2
) / sampling_frequency
preview_waveform = starccato_supernova(
    time_array=preview_time,
    luminosity_distance=injection_parameters["luminosity_distance"],
    model=STARCCATO_MODEL,
    latent_parameter_names=LATENT_PARAMETER_NAMES,
    reference_distance=REFERENCE_DISTANCE_KPC,
    strain_scale=REFERENCE_STRAIN_SCALE,
    **{k: v for k, v in injection_parameters.items() if k != "luminosity_distance"},
)["plus"]

plot_waveform_and_psd_comparison(
    ifos=ifos,
    time_array=preview_time,
    waveform=preview_waveform,
    outpath=os.path.join(outdir, "initial_signal_psd_comparison.png"),
    title_prefix="Initial",
)

# Inject signal
ifos.inject_signal(
    waveform_generator=waveform_generator,
    parameters=injection_parameters,
    raise_error=False,
)

plot_waveform_and_psd_comparison(
    ifos=ifos,
    time_array=preview_time,
    waveform=preview_waveform,
    outpath=os.path.join(outdir, "after_injection_psd_comparison.png"),
    title_prefix="After injection",
)

# ----------------------------
# RUN SAMPLER
# ----------------------------
search_waveform_generator = bilby.gw.waveform_generator.WaveformGenerator(
    duration=duration,
    sampling_frequency=sampling_frequency,
    time_domain_source_model=starccato_supernova,
    parameter_conversion=lambda parameters: (parameters, list()),
    waveform_arguments=dict(
        model=STARCCATO_MODEL,
        latent_parameter_names=LATENT_PARAMETER_NAMES,
        reference_distance=REFERENCE_DISTANCE_KPC,
        strain_scale=REFERENCE_STRAIN_SCALE,
    ),
)

priors = bilby.core.prior.PriorDict()
priors["psi"] = injection_parameters["psi"]
priors["luminosity_distance"] = bilby.core.prior.Uniform(2, 20, "luminosity_distance", unit="$kpc$")
priors["intrinsic_amplitude"] = bilby.core.prior.Uniform(0.5, 1.5, "intrinsic_amplitude")
for name in LATENT_PARAMETER_NAMES:
    priors[name] = bilby.core.prior.Normal(mu=0.0, sigma=1.0, name=name)
priors["ra"] = bilby.core.prior.Uniform(minimum=0, maximum=2 * np.pi, name="ra", boundary="periodic")
priors["dec"] = bilby.core.prior.Sine(name="dec")
priors["geocent_time"] = bilby.core.prior.Uniform(
    injection_parameters["geocent_time"] - 1,
    injection_parameters["geocent_time"] + 1,
    "geocent_time",
    unit="$s$",
)

likelihood = bilby.gw.likelihood.GravitationalWaveTransient(
    interferometers=ifos, waveform_generator=search_waveform_generator
)

result = bilby.run_sampler(
    likelihood=likelihood,
    priors=priors,
    sampler="emcee",
    outdir=outdir,
    label=label,
    npoints=1000,
)

# ----------------------------
# POSTERIOR PSD COMPARISON
# ----------------------------
posterior_samples = result.posterior.sample(1000, random_state=123)
plot_waveform_and_psd_comparison(
    ifos=ifos,
    time_array=preview_time,
    waveform=preview_waveform,
    outpath=os.path.join(outdir, "posterior_psd_comparison.png"),
    title_prefix="Posterior",
    posterior_samples=posterior_samples,
    waveform_generator=search_waveform_generator,
    credible_interval=0.9,
)


23:33 bilby INFO    : Waveform generator instantiated: WaveformGenerator(duration=0.125, sampling_frequency=4096.0, start_time=0, frequency_domain_source_model=None, time_domain_source_model=__main__.starccato_supernova, parameter_conversion=__main__.<lambda>, waveform_arguments={'model': StarccatoVAE(z-dim=32), 'latent_parameter_names': ['z_0', 'z_1', 'z_2', 'z_3', 'z_4', 'z_5', 'z_6', 'z_7', 'z_8', 'z_9', 'z_10', 'z_11', 'z_12', 'z_13', 'z_14', 'z_15', 'z_16', 'z_17', 'z_18', 'z_19', 'z_20', 'z_21', 'z_22', 'z_23', 'z_24', 'z_25', 'z_26', 'z_27', 'z_28', 'z_29', 'z_30', 'z_31'], 'reference_distance': 10.0, 'strain_scale': 1e-21})
23:33 bilby INFO    : Generation of mass parameters failed with message 'mass_1'
23:33 bilby INFO    : Generation of spin parameters failed with message 'spin_1z'
23:33 bilby INFO    : Injected signal in H1:
23:33 bilby INFO    :   optimal SNR = 140.84
23:33 bilby INFO    :   matched filter SNR = 143.30-0.64j
23:33 bilby INFO    :   luminosity_distance = 7.0

  0%|          | 0/100 [00:00<?, ?it/s]

23:41 bilby INFO    : Checkpointing sampler to file outdir/emcee_supernova/sampler.pickle
23:41 bilby INFO    : Max autocorr time = 86
23:41 bilby INFO    : Discarding 258 steps for burn-in


SamplerError: The run has finished, but the chain is not burned in: `nburn < nsteps` (258 < 100). Try increasing the number of steps.